In [1]:
import numpy as np
import cv2


def readFromFile(file, dtype=None, cols=(0,0)):
    f=open(file, "r")
    input = f.readlines()
    contents=[]
    if cols==(0,0):
        for line in input: 
            line=line[:-1]
            contents.append(line.split(' '))
    else:
        for line in input: 
            line=line[:-1]
            contents.append(line.split(' ')[cols[0]:cols[1]])

    if dtype!=None:
        contents = np.array(contents).astype(dtype)
    else:
        contents = np.array(contents)
    return contents


In [2]:
truth = readFromFile("./CUB_200_2011/image_class_labels.txt", 'int16', (1,2))
truth = truth.reshape((1,11788))
truth = truth[0]
print(truth)




[  1   1   1 ... 200 200 200]


In [62]:

filenames = readFromFile("./CUB_200_2011/images.txt",None,(1,2))
class_nms = readFromFile("./CUB_200_2011/classes.txt")
train_idxs = readFromFile("./CUB_200_2011/train_test_split.txt", 'int16',(1,2))
importance = readFromFile("./CUB_200_2011/attributes/class_attribute_labels_continuous.txt", 'float')
attributes = readFromFile("./CUB_200_2011/attributes/image_attribute_labels.txt", 'int16', (0,3))
attribs = np.empty([11788, 312], dtype='int16')
for line in attributes:
    attribs[line[0]-1][line[1]-1] = line[2]
# print(attributes)

[[    1     1     0]
 [    1     2     0]
 [    1     3     0]
 ...
 [11788   310     0]
 [11788   311     0]
 [11788   312     1]]


In [89]:
import random
r1 = random.randint(3, 10) 
imps = np.copy(importance)
y = 0
# topXfeatures = [[None for _ in range(x)] for _ in range(200)]
x=10
topXfeatures = np.empty((200,x), dtype='int')



for row in imps: 
    for j in range(x):
        max_index = np.argmax(row)
        topXfeatures[y,j] = max_index
        row[max_index]=0
#         print(max_index, end = ' ')
#     print()
    y+=1


# topXfeatures[0][2] = 5   
# print(topXfeatures)
# print(topXfeatures.shape)

output = np.full(11788,-1)
n=0
#     print(attribs.shape)
for bird in attribs:
    m=1
    for row in topXfeatures:
        true_total = 0
        for index in row:
            if bird[index]==1:
                true_total+=1
        if true_total>x-4:
            output[n] = m
        m+=1
    n+=1

print(output)


correct = 0
negones = 0
TOTAL = 11788
for i in range(output.shape[0]):
    if output[i]==truth[i]:
        correct+=1
    elif output[i]==-1:
        negones+=1
print(correct/TOTAL)
print(negones)

(11788, 312)
[ -1 171  -1 ... 200 174 200]
0.04691211401425178
1055
(11788, 312)
[ -1 179  46 ... 182 130 198]
0.047675602307431286
1823
(11788, 312)
[ -1  32  -1 ...  12  27 200]
0.035374957583983714
3936
(11788, 312)


KeyboardInterrupt: 

In [87]:
np.random.randint(200, size=11788)

0.017984390906006106
2987


In [95]:



# print(attribs)
# for row in attribs:
# print(importance)
# attributes = np.multiply()
attributes = []
for i in range(11788):
    attributes.append(np.multiply(attribs[i],importance[truth[i]-1]))
    pass
#     if train_idxs[i][0] == 1:
        
#     else:
#         attributes.append(attribs[i])
attributes = np.asarray(attributes)/100
# attributes = np.append(attributes, np.ones((11788,1)))
# attributes.shape = (11788, 313)
# attributes = attributes + np.ones((11788, 1))
# print(attributes)
# print(attributes.shape)
    #multiplys attribute row with importance of 
# def computeLinear():
#     global W
#     W = np.dot(np.linalg.inv(np.dot(np.transpose(attribs), attribs)), np.dot(np.transpose(attribs),truth))
#     pred_rates = np.dot(attribs, np.transpose(W))
#     N = truth.size
#     rmse = np.sqrt(np.sum(np.square(pred_rates-truth))/N)
#     return rmse

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import cluster


[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.21428571 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.10687023 0.        ]
 [0.         0.         0.         ... 0.         0.10687023 0.        ]
 [0.         0.         0.         ... 0.         0.         0.29770992]]
(11788, 312)


In [103]:


atr_train = []
atr_test = []
truth_train = []
truth_test = []
for i in range(11788):
    if train_idxs[i][0] == 1:
        atr_train.append(attributes[i])
        truth_train.append(truth[i])
    else:
        atr_test.append(attributes[i])
        truth_test.append(truth[i])
        
atr_train = np.asarray(atr_train)
atr_test = np.asarray(atr_test)
truth_train = np.asarray(truth_train)
truth_test = np.asarray(truth_test)
    
lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)
lda.fit(atr_train, truth_train)
lda_score_train = lda.score(atr_train, truth_train)
lda_score_test = lda.score(atr_test, truth_test)

print(lda_score_train)
print(lda_score_test)
    



qda = QuadraticDiscriminantAnalysis(store_covariance=True)
qda.fit(atr_train, truth_train)
qda_score_train = qda.score(atr_train, truth_train)
qda_score_test = qda.score(atr_test, truth_test)
print(qda_score_train)
print(qda_score_test)
randarr = np.random.randint(2, size=truth_test.shape[0])

log = LogisticRegression(random_state=0).fit(atr_train, truth_train)
log_score_train = log.score(atr_train, truth_train)
log_score_test = log.score(atr_test, truth_test)

print(log_score_train)
print(log_score_test)

0.8008008008008008
0.7026234035208837


/Users/chrisnelson/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


1.0
0.012081463583016915


/Users/chrisnelson/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/chrisnelson/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.9002335669002336
0.7823610631687953


In [100]:
dt = tree.DecisionTreeClassifier()
dt.fit(atr_train, truth_train)
dt_score_train = dt.score(atr_train, truth_train)
dt_score_test = dt.score(atr_test, truth_test)
print(dt_score_train)
print(dt_score_test)

1.0
0.9216430790472903


In [101]:
from sklearn.svm import SVC
svm = SVC(gamma='auto')
svm.fit(atr_train, truth_train)
SVC(gamma='auto')
print(svm.score(atr_train, truth_train))
print(svm.score(atr_test, truth_test))

0.66332999666333
0.6051087331722471


In [28]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca = PCA(n_components=4)
# x = StandardScaler().fit_transform(atr_train)
principalComponents = pca.fit_transform(atr_train)
# principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
print(pca.explained_variance_ratio_)

[0.12982618 0.10220634 0.07062515 0.05359489]


In [8]:
km = KMeans(200)
km.fit(a)

NameError: name 'KMeans' is not defined

In [ ]:
print(truth_test)

In [ ]:
init_path = "./CUB_200_2011/images/"
images = []
print("hello")

for i in range(10):# in filenames:
    images.append(cv2.imread(init_path+filenames[i][1]))
    
    

print("hello")
images = np.asarray(images)
print(images)

In [ ]:
np.empty([3, 2], dtype='int16')